# Test Case LVV-T2219
The objective of this test case is to verify the ComCam communication interface with SAL by testing the commands, events and telemetry defined by the latest version of the XML. This test case will exercise the functionality of the ComCam on the 3rd level of the Summit and meets the following criteria:

    Only requires the ComCam to be operational
    Does not require any communication with other subsystems
    Does require the use of SAL and the EFD
    


## Setup

In [1]:
from lsst.ts import utils

# Extract your name from the Jupyter Hub
__executed_by__ = os.environ["JUPYTERHUB_USER"]  

# Extract execution date
__executed_on__ = utils.astropy_time_from_tai_unix(utils.current_tai())
__executed_on__.format = "isot"

# This is used later to define where Butler stores the images
summit = os.environ["LSST_DDS_PARTITION_PREFIX"] == "summit"

print(f"\nExecuted by {__executed_by__} on {__executed_on__}."
      f"\n  At the summit? {summit}")


Executed by isotuela on 2022-07-08T16:54:32.642.
  At the summit? False


In [2]:
from lsst.ts import salobj
import logging
import asyncio
import os
import yaml

import numpy as np
from matplotlib import pyplot as plt
from astropy.time import Time
from datetime import datetime, timedelta
import pandas as pd
import astropy.units as u

from lsst.ts.observing.utilities.decorated_logger import DecoratedLogger
from lsst_efd_client import EfdClient

from lsst.ts.observatory.control.maintel import ComCam

The following block sets the necessary environment variables for setting up the DDS/SAL communication

In [3]:
os.environ["LSST_DDS_HISTORYSYNC"] = "30"

Setting up logger

In [4]:
logger = DecoratedLogger.get_decorated_logger()
logger.level = logging.DEBUG

Getting unique index for Script

In [5]:
logger.info(f'Your UID is {os.getuid()}')
index = os.getuid() * 10 + np.random.randint(0, 9)

logger.info(f'The generated index is {index}')

Make sure DDS Daemon is running and startup Domain

In [6]:
domain = salobj.Domain()

Instantiate script for logging into EFD and start script task

In [7]:
test_message = "LVV-T2219 ComCam Interface to SAL"
script = salobj.Controller("Script", index=index)
await script.start_task

EFD setup

In [8]:
#client = EfdClient('ncsa_teststand_efd')
client = EfdClient('summit_efd')

ComCam Initialization

In [9]:
comcam = ComCam(domain=domain)
comcam.set_rem_loglevel(40)

In [10]:
await comcam.start_task

[None, None, None]

In [19]:
await comcam.enable()

Helper functions: Event callback and telemetry check

In [11]:
def evt_time_callback(evt):
    """Print event and time when it is received."""
    time_evt = datetime.utcfromtimestamp(evt.private_sndStamp).strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
    evt_name = str(evt.__class__).split('_')[2]
    print(f'\n {evt_name} logevent at {time_evt} is \n \t{evt}')

In [12]:
async def check_telemetry(topic):
    tel = await client.select_top_n(topic,'*',num=1)
    time_tel = datetime.utcfromtimestamp(tel['private_sndStamp'][0]).strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]

    print(f'Last {topic} telemetry was at {time_tel} UTC \n \tvs time now is {Time.now()}')

Publish in the EFD that test is starting

In [13]:
script.log.info(f'START- {test_message} -- at {Time(datetime.now())}')

--- 
LVV-T2239 (1.0) ComCam Startup Procedure

---
Verify the CCCamera_logevent_availableFilters event is published to the EFD. 
Note: During the successful start up of the system, this event will be published automatically.

In [20]:
available_filters= comcam.rem.cccamera.evt_availableFilters.get()
time_available_filters=  datetime.utcfromtimestamp(available_filters.private_sndStamp).strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
logger.info(f'Available filters at {time_available_filters} are {available_filters.filterNames}')

---
Initially verify the CCCamera_logevent_ccsCommandState event is publishing to the EFD. As there are no commands at the moment, the CCCamera_logevent_ccsCommandState event publishes IDLE.

In [21]:
command_state= comcam.rem.cccamera.evt_ccsCommandState.get()
time_command_state = datetime.utcfromtimestamp(command_state.private_sndStamp).strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
logger.info(f'Current Command State Substate at {time_command_state} is \n {command_state}')

---
While the ccsCommandState is in IDLE, send a CCCamera_command_enableCalibration command.

Note: By default, the CalibrationDetailedState should be in DISABLED. If the CalibrationDetailedState is already in ENABLED, this command will not be accepted.

    Results:
    The CCCamera_logevent_ccsCommandState event publishes BUSY
    The CCCamera_logevent_calibrationDetailedState publishes ENABLED.
    The The CCCamera_logevent_ccsConfigured event is published to the EFD noting the time at which the state transition occurred. ** This logevent does not exist, instead comcam.rem.cccamera.logevent_configurationApplied

In [22]:
comcam.rem.cccamera.evt_calibrationDetailedState.callback = evt_time_callback
comcam.rem.cccamera.evt_ccsCommandState.callback = evt_time_callback   

In [23]:
# Enable Calibration
await comcam.rem.cccamera.cmd_enableCalibration.set_start()


 ccsCommandState logevent at 2022-07-08 16:55:36.290 is 
 	private_revCode: d8de8c36, private_sndStamp: 1657299336.290601, private_rcvStamp: 1657299336.2911742, private_seqNum: 59, private_identity: CCCamera, private_origin: 35289769, substate: 2, timestampTransition: 0.0

 calibrationDetailedState logevent at 2022-07-08 16:55:36.290 is 
 	private_revCode: a610ea82, private_sndStamp: 1657299336.2906916, private_rcvStamp: 1657299336.291222, private_seqNum: 6, private_identity: CCCamera, private_origin: 35289769, substate: 2, timestampTransition: 0.0

 ccsCommandState logevent at 2022-07-08 16:55:36.292 is 
 	private_revCode: d8de8c36, private_sndStamp: 1657299336.2922888, private_rcvStamp: 1657299336.292639, private_seqNum: 60, private_identity: CCCamera, private_origin: 35289769, substate: 1, timestampTransition: 0.0


In [24]:
configuration_applied = comcam.rem.cccamera.evt_configurationApplied.get()
time_configuration_applied = datetime.utcfromtimestamp(configuration_applied.private_sndStamp).strftime(
    "%Y-%m-%d %H:%M:%S.%f")[:-3]
logger.info(f'Current configuration Applied at {time_configuration_applied} is \n {configuration_applied}')

In [25]:
comcam.rem.cccamera.evt_calibrationDetailedState.callback = None
comcam.rem.cccamera.evt_ccsCommandState.callback = None   

---
While the CalibrationDetailedState is in ENABLED,, send a CCCamera_command_clear command.

    Results:
    The command is accepted
    The CCCamera_logevent_RaftsDetailedState event published to the EFD shows it is in the CLEARING state.

In [26]:
comcam.rem.cccamera.evt_raftsDetailedState.callback = evt_time_callback

In [34]:
cmd = await comcam.rem.cccamera.cmd_clear.set_start(nClears=1)
logger.info(cmd)

AckError: msg='Command failed', ackcmd=(ackcmd private_seqNum=47224008, ack=<SalRetCode.CMD_FAILED: -302>, error=0, result='Error : Timeout waiting for state: QUIESCENT')

In [35]:
comcam.rem.cccamera.evt_raftsDetailedState.callback = None

---
Verify the CCCamera_logevent_focalPlaneSummaryInfo event is published to the EFD.

In [36]:
focal_plane_summary = await comcam.rem.cccamera.evt_focalPlaneSummaryInfo.aget()
time_focal_plane_summary = datetime.utcfromtimestamp(focal_plane_summary.private_sndStamp).strftime("%Y-%m-%d %H:%M:%S")
logger.info(f'Focal plane summary at {time_focal_plane_summary}' 
            f' is \n {focal_plane_summary}')

---
Image Test
While the CalibrationDetailedState is in ENABLED, send a CCCamera_command_startImage command with the following parameters:

                        shutter: True
                        sensors
                        keyValueMap
                        obsNote: Any arbitrary string
                        timeout: 30s
    
    
    Results: 
    The CCCamera_logevent_calibrationDetailedState event publishes INTEGRATING
    The CCCamera_logevent_shutterDetailedState event publishes to the EFD updating froom CLOSED to OPENING to OPEN
    A CCCamera_logevent_endShutterOpen event is published at the time the shutter opens
    A CCCamera_logevent_startReadout event is published
    
Beware: The next two steps (discardRows and endImage) need to be run while the camera is INTEGRATING (30 s timeout)

In [71]:
comcam.rem.cccamera.evt_calibrationDetailedState.callback = evt_time_callback
comcam.rem.cccamera.evt_shutterDetailedState.callback = evt_time_callback
comcam.rem.cccamera.evt_endShutterOpen.callback = evt_time_callback
comcam.rem.cccamera.evt_startReadout.callback = evt_time_callback

In [72]:
keyValueMap = f'groupId:{Time.now().isot},imageType:ENGTEST'

In [73]:
# Set parameters
comcam.rem.cccamera.cmd_startImage.set(shutter=True, keyValueMap=keyValueMap, timeout=30)

True

In [74]:
# Start image
await comcam.rem.cccamera.cmd_startImage.start()


 shutterDetailedState logevent at 2022-07-08 17:05:24.973 is 
 	private_revCode: 0ebe7cbe, private_sndStamp: 1657299924.973641, private_rcvStamp: 1657299924.9741132, private_seqNum: 32, private_identity: CCCamera, private_origin: 35289769, substate: 4, timestampTransition: 0.0


----
While the CalibrationDetailedState is in INTEGRATING, send a CCCamera_command_discardRows command of 2 rows.

In [75]:
cmd = await comcam.rem.cccamera.cmd_discardRows.set_start(2)
logger.info(cmd)


 calibrationDetailedState logevent at 2022-07-08 17:05:24.975 is 
 	private_revCode: a610ea82, private_sndStamp: 1657299924.9755883, private_rcvStamp: 1657299924.9758685, private_seqNum: 15, private_identity: CCCamera, private_origin: 35289769, substate: 3, timestampTransition: 0.0


--- 
Before the specified timeout, send a CCCamera_command_endImage command.

Note: If the CCCamera_command_endImage command isn't sent before the specified timeout parameter of the previous command, the shutter will close and the command will not be accepted.

    Results:
    The shutters close 
    A CCCamera_logevent_imageReadoutParameters event is published
    The CCCamera_logevent_shutterDetailedState event publishes to the EFD updating froom OPEN to CLOSING to CLOSED
    A CCCamera_logevent_endShutterClose event is published at the time the shutter close
    A CCCamera_logevent_endReadout event is published

In [76]:
comcam.rem.cccamera.evt_imageReadoutParameters.callback = evt_time_callback
comcam.rem.cccamera.evt_shutterDetailedState.callback = evt_time_callback
comcam.rem.cccamera.evt_endShutterClose.callback = evt_time_callback
comcam.rem.cccamera.evt_endReadout.callback = evt_time_callback

In [77]:
await comcam.rem.cccamera.cmd_endImage.start()


 shutterDetailedState logevent at 2022-07-08 17:05:25.066 is 
 	private_revCode: 0ebe7cbe, private_sndStamp: 1657299925.0662766, private_rcvStamp: 1657299925.066567, private_seqNum: 33, private_identity: CCCamera, private_origin: 35289769, substate: 3, timestampTransition: 0.0

 shutterDetailedState logevent at 2022-07-08 17:05:25.954 is 
 	private_revCode: 0ebe7cbe, private_sndStamp: 1657299925.9541447, private_rcvStamp: 1657299925.954458, private_seqNum: 34, private_identity: CCCamera, private_origin: 35289769, substate: 2, timestampTransition: 0.0

 endShutterClose logevent at 2022-07-08 17:05:26.047 is 
 	private_revCode: 1a6c8bc0, private_sndStamp: 1657299926.0471053, private_rcvStamp: 1657299926.0474405, private_seqNum: 8, private_identity: CCCamera, private_origin: 35289769, timestampTransition: 1657299926.045618

 shutterDetailedState logevent at 2022-07-08 17:05:26.048 is 
 	private_revCode: 0ebe7cbe, private_sndStamp: 1657299926.0488815, private_rcvStamp: 1657299926.0492537,

In [78]:
comcam.rem.cccamera.evt_imageReadoutParameters.callback = None
comcam.rem.cccamera.evt_shutterDetailedState.callback = None
comcam.rem.cccamera.evt_endShutterClose.callback = None
comcam.rem.cccamera.evt_endReadout.callback = None

In [79]:
# Closing callbacks from startImage.
comcam.rem.cccamera.evt_calibrationDetailedState.callback = None
comcam.rem.cccamera.evt_shutterDetailedState.callback = None
comcam.rem.cccamera.evt_endShutterOpen.callback = None
comcam.rem.cccamera.evt_startReadout.callback = None

---
Set Filter Test
Send a CCCamera_command_setFilter command for positionFilter1.
    
    Results:
    The CCCamera_logevent_startSetFilter event is published to the EFD with the name of the filter being set
    The CCCamera_logevent_startLoadFilter event is published to the EFD
    The CCCamera_logevent_filterChangerDetailedState event is published to the EFD showing UNLOADED
    The CCCamera_logevent_endLoadFilter event is published to the EFD
    The CCCamera_logevent_filterChangerDetailedState event is published to the EFD and showing ROTATING
    The CCCamera_logevent_startRotateCarousel event is published to the EFD
    The CCCamera_logevent_filterChangerDetailedState event is published to the EFD and showing UNLOADED
    The CCCamera_logevent_endRotatorCarousel event is published to the EFD
    The CCCamera_logevent_filterChangerDetailedState event is published to the EFD and showing LOADING
    The CCCamera_logevent_startLoadFilter event is published to the EFD
    The CCCamera_logevent_filterChangerDetailedState event is published to the EFD showing LOADED
    The CCCamera_logevent_endSetFilter event is published to the EFD

In [89]:
comcam.rem.cccamera.evt_startSetFilter.callback = evt_time_callback
comcam.rem.cccamera.evt_startLoadFilter.callback = evt_time_callback
comcam.rem.cccamera.evt_filterChangerDetailedState.callback = evt_time_callback
comcam.rem.cccamera.evt_endLoadFilter.callback = evt_time_callback
comcam.rem.cccamera.evt_startRotateCarousel.callback = evt_time_callback
comcam.rem.cccamera.evt_endRotateCarousel.callback = evt_time_callback
comcam.rem.cccamera.evt_endSetFilter.callback = evt_time_callback

In [90]:
await comcam.rem.cccamera.cmd_setFilter.set_start(name = 'r_03')


 startSetFilter logevent at 2022-07-08 17:16:08.457 is 
 	private_revCode: 031a81c3, private_sndStamp: 1657300568.4575944, private_rcvStamp: 1657300568.4579308, private_seqNum: 3, private_identity: CCCamera, private_origin: 35289769, filterName: r_03, filterType: r

 filterChangerDetailedState logevent at 2022-07-08 17:16:08.460 is 
 	private_revCode: 00917c89, private_sndStamp: 1657300568.4601815, private_rcvStamp: 1657300568.4605129, private_seqNum: 6, private_identity: CCCamera, private_origin: 35289769, substate: 1, timestampTransition: 0.0

 filterChangerDetailedState logevent at 2022-07-08 17:16:23.458 is 
 	private_revCode: 00917c89, private_sndStamp: 1657300583.4589872, private_rcvStamp: 1657300583.4592266, private_seqNum: 7, private_identity: CCCamera, private_origin: 35289769, substate: 4, timestampTransition: 0.0

 startRotateCarousel logevent at 2022-07-08 17:16:23.459 is 
 	private_revCode: dba0ed3b, private_sndStamp: 1657300583.4598105, private_rcvStamp: 1657300583.46008

In [91]:
comcam.rem.cccamera.evt_startSetFilter.callback = None
comcam.rem.cccamera.evt_startLoadFilter.callback = None
comcam.rem.cccamera.evt_filterChangerDetailedState.callback = None
comcam.rem.cccamera.evt_endLoadFilter.callback = None
comcam.rem.cccamera.evt_startRotateCarousel.callback = None
comcam.rem.cccamera.evt_endRotateCarousel.callback = None
comcam.rem.cccamera.evt_endSetFilter.callback = None

In [92]:
await comcam.get_current_filter()

'r_03'

---
Send a CCCamera_command_setFilter command with NONE.

    Results:
        The CCCamera_logevent_startSetFilter event is published to the EFD with the name of the filter being set
    The CCCamera_logevent_startUnloadFilter event is published to the EFD
        The CCCamera_logevent_filterChangerDetailedState event is published to the EFD showing UNLOADING
    The CCCamera_logevent_endUnloadFilter event is published to the EFD
        The CCCamera_logevent_filterChangerDetailedState event is published to the EFD 
    The CCCamera_logevent_endSetFilter event is published to the EFD

In [95]:
comcam.rem.cccamera.evt_startSetFilter.callback = evt_time_callback
comcam.rem.cccamera.evt_startUnloadFilter.callback = evt_time_callback
comcam.rem.cccamera.evt_filterChangerDetailedState.callback = evt_time_callback
comcam.rem.cccamera.evt_endSetFilter.callback = evt_time_callback

In [101]:
await comcam.rem.cccamera.cmd_setFilter.set_start(name=None)

AckError: msg='Command failed', ackcmd=(ackcmd private_seqNum=1520055166, ack=<SalRetCode.CMD_NOPERM: -300>, error=0, result='Ack : NO Command rejected: Invalid filter: ')

In [102]:
comcam.rem.cccamera.evt_startSetFilter.callback = None
comcam.rem.cccamera.evt_startUnloadFilter.callback = None
comcam.rem.cccamera.evt_filterChangerDetailedState.callback = None
comcam.rem.cccamera.evt_endSetFilter.callback = None

--- 
Take Image Test
While in the ENABLED state, send a CCCamera_command_initImage command of 15s.

Note: This deltaT value is an estimate of time before sending the CCCamera_command_takeImage command. If the command can be sent sooner/later, adjust the deltaT value.

    Results: 
    The command is accepted
    The CCCamera_logevent_ImageReadinesDetailedState event is published to the EFD showing the system transitions into the Getting_Ready state
    A CCCamera_logevent_prepareToTakeImage event is published to the EFD at the time of the transition
    A CCCamera_logevent_NotReadyToTakeImage event is published before reaching the Ready state.
    
    
**** Not available at EFD `comcam.rem.cccamera.evt_prepareToTakeImage` & `comcam.rem.cccamera.evt_notReadyToTakeImage`

In [103]:
comcam.rem.cccamera.evt_imageReadinessDetailedState.callback = evt_time_callback
comcam.rem.cccamera.evt_prepareToTakeImage.callback = evt_time_callback
comcam.rem.cccamera.evt_notReadyToTakeImage.callback = evt_time_callback

In [105]:
await comcam.rem.cccamera.cmd_initImage.set_start(deltaT = 15)


 imageReadinessDetailedState logevent at 2022-07-08 17:21:20.540 is 
 	private_revCode: 90d66834, private_sndStamp: 1657300880.5404873, private_rcvStamp: 1657300880.5408556, private_seqNum: 7, private_identity: CCCamera, private_origin: 35289769, substate: 1, timestampTransition: 0.0



 imageReadinessDetailedState logevent at 2022-07-08 17:21:20.541 is 
 	private_revCode: 90d66834, private_sndStamp: 1657300880.5413995, private_rcvStamp: 1657300880.5417087, private_seqNum: 8, private_identity: CCCamera, private_origin: 35289769, substate: 3, timestampTransition: 0.0


In [106]:
comcam.rem.cccamera.evt_imageReadinessDetailedState.callback = None
comcam.rem.cccamera.evt_prepareToTakeImage.callback = None
comcam.rem.cccamera.evt_notReadyToTakeImage.callback = None

---
Verify the Camera is ready to take a picture.
    
    Results:
    The CCCamera_logevent_ImageReadinesDetailedState event is published to the EFD showing the system transitions into the Ready state
    A CCCamera_logevent_readyToTakeImage event is published to the EFD at the time of the transition
    A CCCamera_logevent_endInitializeImage event is published to the EFD at the time of the transition
    
**** Not available at EFD `comcam.rem.cccamera.evt_readyToTakeImage` & `comcam.rem.cccamera.evt_endInitializeImage`

In [107]:
readiness = comcam.rem.cccamera.evt_imageReadinessDetailedState.get()
time_readiness = datetime.utcfromtimestamp(readiness.private_sndStamp).strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
logger.info(f'Image Readiness Detailed State at {time_readiness}' 
            f' is \n {readiness}')

In [108]:
ready_to_take_image = comcam.rem.cccamera.evt_readyToTakeImage.get()
time_ready_to_take_image = datetime.utcfromtimestamp(ready_to_take_image.private_sndStamp).strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
logger.info(f'Ready to take image at {time_ready_to_take_image}' 
            f' is \n {ready_to_take_image}')


AttributeError: 'NoneType' object has no attribute 'private_sndStamp'

In [109]:
end_initialize_image = comcam.rem.cccamera.evt_endInitializeImage.get()
time_end_initialize_image = datetime.utcfromtimestamp(end_initialize_image.private_sndStamp).strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
logger.info(f'End Initialize Image at {time_end_initialize_image}' 
            f' is \n {end_initialize_image}')

AttributeError: 'NoneType' object has no attribute 'private_sndStamp'

---
With the Camera ready to take a picture, send the CCCamera_command_takeImages command.

    Results:
    A CCCamera_logevent_startIntegration event is published to the EFD
    A CCCamera_logevent_startShutterOpen event is published to the EFD.
    A CCCamera_logevent_endShutterOpen event is published to the EFD.
    The Camera publishes a CCCamera_logevent_startShutterClose event as the camera shutter starts to shut
    The Camera publishes a CCCamera_logevent_endShutterClose event when the camera shutter is closed
    A CCCamera_logevent_startReadout event is published
    A CCCamera_logevent_endReadout event is published

In [110]:
comcam.rem.cccamera.evt_startIntegration.callback = evt_time_callback
comcam.rem.cccamera.evt_startShutterOpen.callback = evt_time_callback
comcam.rem.cccamera.evt_endShutterOpen.callback = evt_time_callback
comcam.rem.cccamera.evt_startShutterClose.callback = evt_time_callback
comcam.rem.cccamera.evt_endShutterClose.callback = evt_time_callback
comcam.rem.cccamera.evt_startReadout.callback = evt_time_callback
comcam.rem.cccamera.evt_endReadout.callback = evt_time_callback

In [111]:
keyValueMap = f'groupId:{Time.now().isot},imageType:ENGTEST'

In [112]:
# Set parameters
comcam.rem.cccamera.cmd_takeImages.set(shutter=True, keyValueMap=keyValueMap, numImages=1, expTime=20, obsNote = 'test')

True

In [113]:
await comcam.rem.cccamera.cmd_takeImages.start()


 startIntegration logevent at 2022-07-08 17:25:19.260 is 
 	private_revCode: ac7ce4ef, private_sndStamp: 1657301119.260018, private_rcvStamp: 1657301119.260282, private_seqNum: 9, private_identity: CCCamera, private_origin: 35289769, additionalKeys: groupId:imageType, additionalValues: 2022-07-08T17\:24\:33.142:ENGTEST, imagesInSequence: 1, imageName: CC_O_20220708_000013, imageIndex: 1, imageSource: CC, imageController: O, imageDate: 20220708, imageNumber: 13, timestampAcquisitionStart: 1657299929.1440973, exposureTime: 20.0, mode: normal, timeout: 20.0

 startShutterOpen logevent at 2022-07-08 17:25:19.261 is 
 	private_revCode: e26069d4, private_sndStamp: 1657301119.2611918, private_rcvStamp: 1657301119.261704, private_seqNum: 8, private_identity: CCCamera, private_origin: 35289769, timestampTransition: 1657301119.2599976

 endShutterOpen logevent at 2022-07-08 17:25:20.242 is 
 	private_revCode: dc9d3865, private_sndStamp: 1657301120.2422593, private_rcvStamp: 1657301120.2426662, 


 endReadout logevent at 2022-07-08 17:25:44.323 is 
 	private_revCode: 82211edd, private_sndStamp: 1657301144.323929, private_rcvStamp: 1657301144.324305, private_seqNum: 9, private_identity: CCCamera, private_origin: 35289769, additionalKeys: groupId:imageType, additionalValues: 2022-07-08T17\:24\:33.142:ENGTEST, imagesInSequence: 1, imageName: CC_O_20220708_000013, imageIndex: 1, imageSource: CC, imageController: O, imageDate: 20220708, imageNumber: 13, timestampAcquisitionStart: 1657299929.1440973, requestedExposureTime: 20.0, timestampEndOfReadout: 1657301144.3218834


In [114]:
comcam.rem.cccamera.evt_startIntegration.callback = None
comcam.rem.cccamera.evt_startShutterOpen.callback = None
comcam.rem.cccamera.evt_endShutterOpen.callback = None
comcam.rem.cccamera.evt_startShutterClose.callback = None
comcam.rem.cccamera.evt_endShutterClose.callback = None
comcam.rem.cccamera.evt_startReadout.callback = None
comcam.rem.cccamera.evt_endReadout.callback = None

---
Verify the Camera is done taking pictures.

    Results:
    A CCCamera_logevent_endTakeImage event is published to the EFD at the time the state transition occurs.
    A CCCamera_logevent_endOfImageTelemetry event is published
    
**** Not available at EFD `comcam.rem.cccamera.evt_endTakeImage` 

In [115]:
end_take_image = comcam.rem.cccamera.evt_endTakeImage.get()
time_end_take_image = datetime.utcfromtimestamp(end_take_image.private_sndStamp).strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
logger.info(f'End Take Image at {time_end_take_image}' 
            f' is \n {end_take_image}')

AttributeError: 'NoneType' object has no attribute 'private_sndStamp'

In [116]:
end_of_image_telemetry = comcam.rem.cccamera.evt_endOfImageTelemetry.get()
time_end_of_image_telemetry = datetime.utcfromtimestamp(end_of_image_telemetry.private_sndStamp).strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
logger.info(f'End of Image Telemetry at {time_end_of_image_telemetry}' 
            f' is \n {end_of_image_telemetry}')

---
Guiders Image Test

While in the ENABLED state, send a CCCamera_command_initImage command of 15s.

    Results:
    The command is accepted
    The CCCamera_logevent_ImageReadinesDetailedState event is published to the EFD showing the system transitions into the Getting_Ready state
    A CCCamera_logevent_prepareToTakeImage event is published to the EFD at the time of the transition
    A CCCamera_logevent_NotReadyToTakeImage event is published before reaching the Ready state.

In [117]:
comcam.rem.cccamera.evt_imageReadinessDetailedState.callback = evt_time_callback
comcam.rem.cccamera.evt_prepareToTakeImage.callback = evt_time_callback
comcam.rem.cccamera.evt_notReadyToTakeImage.callback = evt_time_callback

In [118]:
await comcam.rem.cccamera.cmd_initImage.set_start(deltaT = 15)


 imageReadinessDetailedState logevent at 2022-07-08 17:27:38.390 is 
 	private_revCode: 90d66834, private_sndStamp: 1657301258.3901618, private_rcvStamp: 1657301258.3904345, private_seqNum: 9, private_identity: CCCamera, private_origin: 35289769, substate: 1, timestampTransition: 0.0

 imageReadinessDetailedState logevent at 2022-07-08 17:27:38.391 is 
 	private_revCode: 90d66834, private_sndStamp: 1657301258.3911722, private_rcvStamp: 1657301258.3914313, private_seqNum: 10, private_identity: CCCamera, private_origin: 35289769, substate: 3, timestampTransition: 0.0


In [119]:
comcam.rem.cccamera.evt_imageReadinessDetailedState.callback = None
comcam.rem.cccamera.evt_prepareToTakeImage.callback = None
comcam.rem.cccamera.evt_notReadyToTakeImage.callback = None

---
Verify the Camera is ready to take a picture.

    Results:
    The CCCamera_logevent_ImageReadinesDetailedState event is published to the EFD showing the system transitions into the Ready state
    A CCCamera_logevent_readyToTakeImage event is published to the EFD at the time of the transition
    A CCCamera_logevent_endInitializeImage event is published to the EFD at the time of the transition

In [120]:
readiness = comcam.rem.cccamera.evt_imageReadinessDetailedState.get()
time_readiness = datetime.utcfromtimestamp(readiness.private_sndStamp).strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
logger.info(f'Image Readiness Detailed State at {time_readiness}' 
            f' is \n {readiness}')

In [121]:
ready_to_take_image = comcam.rem.cccamera.evt_readyToTakeImage.get()
time_ready_to_take_image = datetime.utcfromtimestamp(ready_to_take_image.private_sndStamp).strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
logger.info(f'Ready to take image at {time_ready_to_take_image}' 
            f' is \n {ready_to_take_image}')

AttributeError: 'NoneType' object has no attribute 'private_sndStamp'

In [122]:
end_initialize_image = comcam.rem.cccamera.evt_endInitializeImage.get()
time_end_initialize_image = datetime.utcfromtimestamp(end_initialize_image.private_sndStamp).strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
logger.info(f'End Initialize Image at {time_end_initialize_image}' 
            f' is \n {end_initialize_image}')

AttributeError: 'NoneType' object has no attribute 'private_sndStamp'

---
Send a CCCamera_command_initGuider command.

    Results:
    An CCCamera_logevent_endInitializeGuider event is published to the EFD.

**** Missing at EFD `comcam.rem.cccamera.evt_endInitializeGuider`

In [123]:
comcam.rem.cccamera.evt_endInitializeGuider.callback = evt_time_callback

In [124]:
await comcam.rem.cccamera.cmd_initGuiders.start()

In [125]:
comcam.rem.cccamera.evt_endInitializeGuider.callback = None

---
With the Camera ready to take a picture, send the CCCamera_command_takeImages command.

    Results:
    A CCCamera_logevent_startIntegration event is published to the EFD
    A CCCamera_logevent_startShutterOpen event is published to the EFD.
    A CCCamera_logevent_endShutterOpen event is published to the EFD.
    The Camera publishes a CCCamera_logevent_startShutterClose event as the camera shutter starts to shut
    The Camera publishes a CCCamera_logevent_endShutterClose event when the camera shutter is closed
    A CCCamera_logevent_startReadout event is published
    A CCCamera_logevent_endReadout event is published

In [126]:
comcam.rem.cccamera.evt_startIntegration.callback = evt_time_callback
comcam.rem.cccamera.evt_startShutterOpen.callback = evt_time_callback
comcam.rem.cccamera.evt_endShutterOpen.callback = evt_time_callback
comcam.rem.cccamera.evt_startShutterClose.callback = evt_time_callback
comcam.rem.cccamera.evt_endShutterClose.callback = evt_time_callback
comcam.rem.cccamera.evt_startReadout.callback = evt_time_callback
comcam.rem.cccamera.evt_endReadout.callback = evt_time_callback

In [127]:
keyValueMap = f'groupId:{Time.now().isot},imageType:ENGTEST'

In [128]:
# Set parameters
comcam.rem.cccamera.cmd_takeImages.set(shutter=True, keyValueMap=keyValueMap, numImages=1, expTime=20, obsNote = 'test')

True

In [129]:
await comcam.rem.cccamera.cmd_takeImages.start()


 startIntegration logevent at 2022-07-08 17:30:20.866 is 
 	private_revCode: ac7ce4ef, private_sndStamp: 1657301420.8659997, private_rcvStamp: 1657301420.8663218, private_seqNum: 10, private_identity: CCCamera, private_origin: 35289769, additionalKeys: groupId:imageType, additionalValues: 2022-07-08T17\:29\:39.646:ENGTEST, imagesInSequence: 1, imageName: CC_O_20220708_000014, imageIndex: 1, imageSource: CC, imageController: O, imageDate: 20220708, imageNumber: 14, timestampAcquisitionStart: 1657301144.319789, exposureTime: 20.0, mode: normal, timeout: 20.0

 startShutterOpen logevent at 2022-07-08 17:30:20.868 is 
 	private_revCode: e26069d4, private_sndStamp: 1657301420.8680701, private_rcvStamp: 1657301420.8684192, private_seqNum: 9, private_identity: CCCamera, private_origin: 35289769, timestampTransition: 1657301420.8666835

 endShutterOpen logevent at 2022-07-08 17:30:21.848 is 
 	private_revCode: dc9d3865, private_sndStamp: 1657301421.8486185, private_rcvStamp: 1657301421.848989

In [130]:
comcam.rem.cccamera.evt_startIntegration.callback = None
comcam.rem.cccamera.evt_startShutterOpen.callback = None
comcam.rem.cccamera.evt_endShutterOpen.callback = None
comcam.rem.cccamera.evt_startShutterClose.callback = None
comcam.rem.cccamera.evt_endShutterClose.callback = None
comcam.rem.cccamera.evt_startReadout.callback = None
comcam.rem.cccamera.evt_endReadout.callback = None

---
Verify the Camera is done taking pictures.

    Results:
    A CCCamera_logevent_endTakeImage event is published to the EFD at the time the state transition occurs.
    A CCCamera_logevent_endOfImageTelemetry event is published

In [131]:
end_take_image = comcam.rem.cccamera.evt_endTakeImage.get()
time_end_take_image = datetime.utcfromtimestamp(end_take_image.private_sndStamp).strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
logger.info(f'End Take Image at {time_end_take_image}' 
            f' is \n {end_take_image}')

AttributeError: 'NoneType' object has no attribute 'private_sndStamp'

In [132]:
end_of_image_telemetry = comcam.rem.cccamera.evt_endOfImageTelemetry.get()
time_end_of_image_telemetry = datetime.utcfromtimestamp(end_of_image_telemetry.private_sndStamp).strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
logger.info(f'End of Image Telemetry at {time_end_of_image_telemetry}' 
            f' is \n {end_of_image_telemetry}')

---
21 Verify the CCCamera_logevent_imageVisualization event is published into the EFD.    

In [133]:
image_visualization = comcam.rem.cccamera.evt_imageVisualization.get()
time_image_visualization = datetime.utcfromtimestamp(image_visualization.private_sndStamp).strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
logger.info(f'Image Visualization at {time_image_visualization}' 
            f' is \n {image_visualization}')

AttributeError: 'NoneType' object has no attribute 'private_sndStamp'

---
Verify the CCCamera_logevent_imageStored event is published into the EFD.

In [134]:
image_stored = comcam.rem.cccamera.evt_imageStored.get()
time_image_stored = datetime.utcfromtimestamp(image_stored.private_sndStamp).strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
logger.info(f'Image Stored at {time_image_stored}' 
            f' is \n {image_stored}')

AttributeError: 'NoneType' object has no attribute 'private_sndStamp'

In [166]:
await comcam.get_current_filter()

'r_03'

In [167]:
comcam.rem.cccamera.evt_startSetFilter.callback = evt_time_callback
comcam.rem.cccamera.evt_endSetFilter.callback = evt_time_callback

In [168]:
comcam.rem.cccamera.cmd_setFilter.set_start(name = 'i_06')

<coroutine object RemoteCommand.set_start at 0x7fa1cc278900>

---
Before an CCCamera_logevent_endSetFilter event is published, send a CCCamera_command_stop command.

Note: The CCCamera_command_setFilter command will stop once it is safe to do so. The CCCamera_command_stop command may not stop the filter from being set if the command was given while in an unsafe position.

    Results:
    The command is accepted
    The CCCamera_logevent_endSetFilter event shows that no filter has been set

In [174]:
comcam.rem.cccamera.cmd_stop

AckError: msg='Command failed', ackcmd=(ackcmd private_seqNum=2007594315, ack=<SalRetCode.CMD_NOPERM: -300>, error=0, result='Ack : NO Command rejected: Unsupported command StopCommand{}')

In [170]:
comcam.rem.cccamera.evt_startSetFilter.callback = None
comcam.rem.cccamera.evt_endSetFilter.callback = None

In [171]:
await comcam.get_current_filter()

'r_03'

---
While the CalibrationDetailedState is in ENABLED, send a CCCamera_command_startImage command with the following parameters:

                        shutter: True
                        sensors
                        keyValueMap
                        obsNote: Any arbitrary string
                        timeout: 30s
    

In [175]:
# Check CalibrationDetailedState is ENABLED
calibration_detailed_state= comcam.rem.cccamera.evt_calibrationDetailedState.get()
time_calibration_detailed_state=  datetime.utcfromtimestamp(calibration_detailed_state.private_sndStamp).strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
logger.info(f'Calibration Detailed State at {time_calibration_detailed_state} are {calibration_detailed_state}')

In [176]:
keyValueMap = f'groupId:{Time.now().isot},imageType:ENGTEST'

In [177]:
# Set parameters
comcam.rem.cccamera.cmd_startImage.set(shutter=True, keyValueMap=keyValueMap, timeout=30)

True

In [190]:
# Start image
await comcam.rem.cccamera.cmd_startImage.start()

---
Before the specified timeout, send a CCCamera_command_stop command.

    Results:
    The previous CCCamera_command_startImage command is abandoned.

In [191]:
# Check CalibrationDetailedState is ENABLED
start_integration= comcam.rem.cccamera.evt_startIntegration.get()
time_start_integration=  datetime.utcfromtimestamp(start_integration.private_sndStamp).strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
logger.info(f'Start Integration at {time_start_integration} are {start_integration}')

In [192]:
stop = comcam.rem.cccamera.cmd_stop
logger.info(f'{stop} at {Time.now()}')

In [195]:
end_of_image_telemetry = comcam.rem.cccamera.evt_endOfImageTelemetry.get()
time_end_of_image_telemetry = datetime.utcfromtimestamp(end_of_image_telemetry.private_sndStamp).strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
logger.info(f'End of Image Telemetry at {time_end_of_image_telemetry}' 
            f' is \n {end_of_image_telemetry}')

---
Playlist Test - only valid when using emulated DAQ
Send a CCCamera_command_playList command of the playlist to be defined and the current folder of where the images exist.

    Results:
    The command is accepted and a playlist is created.

In [ ]:
await comcam.rem.cccamera.cmd_playlist.set_start(playlist='tiago-1.playlist')

AckError: msg='Command failed', ackcmd=(ackcmd private_seqNum=1232624188, ack=<SalRetCode.CMD_NOPERM: -300>, error=0, result='Ack : NO Command rejected: Invalid folder: ')

---
Send a CCCamera_command_play coommand of the playlist and an input of true for the repeat parameter.
    
    Results:
    The command is accepted
    The playlist is played and repeats

In [197]:
await comcam.rem.cccamera.cmd_play.set_start(playlist='tiago-1.playlist', repeat =True)

---
With the Summary State of the Camera in ENABLED and the CalibrationDetailedState in ENABLED, send a CCCamera_command_disableCalibration command.

    Results:
    The  CCCamera_logevent_CalibrationDetailedState event publishes DISABLED.

In [198]:
comcam.rem.cccamera.evt_calibrationDetailedState.callback = evt_time_callback

In [199]:
await comcam.rem.cccamera.cmd_disableCalibration.set_start()


 calibrationDetailedState logevent at 2022-07-08 17:46:21.132 is 
 	private_revCode: a610ea82, private_sndStamp: 1657302381.132062, private_rcvStamp: 1657302381.132455, private_seqNum: 23, private_identity: CCCamera, private_origin: 35289769, substate: 1, timestampTransition: 0.0


In [200]:
comcam.rem.cccamera.evt_calibrationDetailedState.callback = None

--- 
Telemetry


Verify the CCCamera_fcs telemetry data is being published to the EFD.

---
Verify the CCCamera_bonn_shutter_Device telemetry data is being published to the EFD.

In [201]:
await check_telemetry('lsst.sal.CCCamera.bonn_shutter_Device')

Last lsst.sal.CCCamera.bonn_shutter_Device telemetry was at 2022-07-08 17:46:31.531 UTC 
 	vs time now is 2022-07-08 17:46:01.306883


---
Verify the CCCamera_daq_monitor_Store telemetry data is being published to the EFD.

In [202]:
await check_telemetry('lsst.sal.CCCamera.daq_monitor_Store')

Last lsst.sal.CCCamera.daq_monitor_Store telemetry was at 2022-07-08 17:46:35.535 UTC 
 	vs time now is 2022-07-08 17:46:01.568242


---
Verify the CCCamera_rebpower_Reb telemetry data is being published to the EFD.

In [203]:
await check_telemetry('lsst.sal.CCCamera.rebpower_Reb')

Last lsst.sal.CCCamera.rebpower_Reb telemetry was at 2022-07-08 17:46:37.992 UTC 
 	vs time now is 2022-07-08 17:46:01.989672


---
Verify the CCCamera_rebpower_Rebps telemetry data is being published to the EFD.

In [204]:
await check_telemetry('lsst.sal.CCCamera.rebpower_Rebps')

Last lsst.sal.CCCamera.rebpower_Rebps telemetry was at 2022-07-06 21:22:33.530 UTC 
 	vs time now is 2022-07-08 17:46:02.248874


---
Verify the CCCamera_vacuum_Cold1 telemetry data is being published to the EFD.

In [205]:
await check_telemetry('lsst.sal.CCCamera.vacuum_Cold1')

Last lsst.sal.CCCamera.vacuum_Cold1 telemetry was at 2022-07-08 17:46:33.172 UTC 
 	vs time now is 2022-07-08 17:46:02.555867


---
Verify the CCCamera_vacuum_Cold2 telemetry data is being published to the EFD.

In [206]:
await check_telemetry('lsst.sal.CCCamera.vacuum_Cold2')

Last lsst.sal.CCCamera.vacuum_Cold2 telemetry was at 2022-07-08 17:46:33.172 UTC 
 	vs time now is 2022-07-08 17:46:02.844443


---
Verify the CCCamera_vacuum_Cryo telemetry data is being published to the EFD.

In [207]:
await check_telemetry('lsst.sal.CCCamera.vacuum_Cryo')

Last lsst.sal.CCCamera.vacuum_Cryo telemetry was at 2022-07-08 17:46:33.172 UTC 
 	vs time now is 2022-07-08 17:46:03.127831


---
Verify the CCCamera_vacuum_IonPumps telemetry data is being published to the EFD.

In [208]:
await check_telemetry('lsst.sal.CCCamera.vacuum_IonPumps')

Last lsst.sal.CCCamera.vacuum_IonPumps telemetry was at 2022-07-08 17:46:33.172 UTC 
 	vs time now is 2022-07-08 17:46:03.374151


---
Verify the CCCamera_vacuum_Rtds telemetry data is being published to the EFD.

In [209]:
await check_telemetry('lsst.sal.CCCamera.vacuum_Rtds')

Last lsst.sal.CCCamera.vacuum_Rtds telemetry was at 2022-07-08 17:46:33.172 UTC 
 	vs time now is 2022-07-08 17:46:03.615487


---
Verify the CCCamera_vacuum_Turbo telemetry data is being published to the EFD.

In [210]:
await check_telemetry('lsst.sal.CCCamera.vacuum_Turbo')

Last lsst.sal.CCCamera.vacuum_Turbo telemetry was at 2022-07-08 17:46:33.172 UTC 
 	vs time now is 2022-07-08 17:46:03.888553


---
Verify the CCCamera_vacuum_VQMonitor telemetry data is being published to the EFD.

In [211]:
await check_telemetry('lsst.sal.CCCamera.vacuum_VQMonitor')

Last lsst.sal.CCCamera.vacuum_VQMonitor telemetry was at 2022-07-08 17:46:33.172 UTC 
 	vs time now is 2022-07-08 17:46:04.157110


---
Verify the CCCamera_quadbox_BFR telemetry data is being published to the EFD.

In [212]:
await check_telemetry('lsst.sal.CCCamera.quadbox_BFR')

Last lsst.sal.CCCamera.quadbox_BFR telemetry was at 2022-07-08 17:46:34.450 UTC 
 	vs time now is 2022-07-08 17:46:04.421820


---
Verify the CCCamera_quadbox_PDU_24VD telemetry data is being published to the EFD.

In [213]:
await check_telemetry('lsst.sal.CCCamera.quadbox_PDU_24VD')

Last lsst.sal.CCCamera.quadbox_PDU_24VD telemetry was at 2022-07-08 17:46:34.450 UTC 
 	vs time now is 2022-07-08 17:46:04.703223


---
Verify the CCCamera_quadbox_PDU_48V telemetry data is being published to the EFD.

In [214]:
await check_telemetry('lsst.sal.CCCamera.quadbox_PDU_48V')

Last lsst.sal.CCCamera.quadbox_PDU_48V telemetry was at 2022-07-08 17:46:34.450 UTC 
 	vs time now is 2022-07-08 17:46:04.992532


---
Verify the CCCamera_quadbox_PDU_5V telemetry data is being published to the EFD.

In [215]:
await check_telemetry('lsst.sal.CCCamera.quadbox_PDU_5V')

Last lsst.sal.CCCamera.quadbox_PDU_5V telemetry was at 2022-07-08 17:46:34.450 UTC 
 	vs time now is 2022-07-08 17:46:05.234315


---
Verify the CCCamera_focal_plane_Ccd telemetry data is being published to the EFD.

In [216]:
await check_telemetry('lsst.sal.CCCamera.focal_plane_Ccd')

Last lsst.sal.CCCamera.focal_plane_Ccd telemetry was at 2022-07-08 17:46:37.992 UTC 
 	vs time now is 2022-07-08 17:46:05.519602


---
Verify the CCCamera_focal_plane_RebTotalPower telemetry data is being published to the EFD.

In [217]:
await check_telemetry('lsst.sal.CCCamera.focal_plane_RebTotalPower')

Last lsst.sal.CCCamera.focal_plane_RebTotalPower telemetry was at 2022-07-08 17:46:34.866 UTC 
 	vs time now is 2022-07-08 17:46:05.758352


---
Verify the CCCamera_focal_plane_Segment telemetry data is being published to the EFD.

In [218]:
await check_telemetry('lsst.sal.CCCamera.focal_plane_Segment')

Last lsst.sal.CCCamera.focal_plane_Segment telemetry was at 2022-07-08 17:46:37.991 UTC 
 	vs time now is 2022-07-08 17:46:06.129268


In [219]:
script.log.info(f"STOP- {test_message}")